In [1]:
from pyts.datasets import load_gunpoint
import matplotlib.pyplot as plt
import numpy as np
import mass_ts as mts

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn import svm
from sklearn.metrics import f1_score

import scipy.stats as sps

import pandas as pd

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from scipy.special import entr

/Users/stefan-temp/Desktop/General/Privat/greedy-shapelet-search/venv-hyp-mar/lib/python3.9/site-packages/mass_ts/_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


In [2]:
X_train, X_test, y_train, y_test = load_gunpoint(return_X_y=True)

In [6]:
class GreedyShapeletSearch():
    def __init__(self):
        self.shapelets = []
        self.exclusion_zone = {}
        self.features = []
        self.top_shapelets = []

    @staticmethod
    def rolling_window(a, window):
        shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
        strides = a.strides + (a.strides[-1],)
        return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)
    
    def get_candidate_mins(self, sample_data, shapelet_size = 10):
        """
        Function that calculates the distance of all candidates of a given data set to all all other candidates.
        CAREFUL:
        - memory blows up quickly.
        - contains the zeros (distance of candidates to itself)
        """
        # Window the array
        windowed_data = self.rolling_window(sample_data, shapelet_size)
        # Standardize candidates
        windowed_data = self.standardize_samples_candidates(windowed_data)
        distances = []
        for sample_candidates in windowed_data:
            candidate_distances = np.array([((windowed_data - candidate)**2).sum(axis=-1).min(axis=-1) for candidate in sample_candidates])
            distances.append(candidate_distances)
        return np.stack(distances)
    
    def get_top_k_shapelets(self, X_train, y_train, scoring_function, n_shapelets=1, shapelet_min_size = 10, shapelet_max_size=20):

        for _ in range(n_shapelets):
            self.shapelets = []
            self.main_event_loop(X_train, y_train, scoring_function, shapelet_min_size = shapelet_min_size, shapelet_max_size = shapelet_max_size)

    def main_event_loop(self, X_train, y_train, scoring_function, shapelet_min_size = 30, shapelet_max_size = 31):
        """
        The main event loop contains the series of steps required for the algorithm.
        """
        for shapelet_size in range(shapelet_min_size, shapelet_max_size):
            # Calculate all of the candidate minimums throughout the dataset - shape: n_samples, n_samples, n_candidate
            profiles = self.get_candidate_mins(X_train, shapelet_size)
            # Extract a shapelet for n_shapelets
            self.evaluate_candidates(profiles, y_train, shapelet_size, scoring_function)
        # Retrieve top shapelets
        self.retrieve_top_shapelet(X_train)
    
    def evaluate_candidates(self, profiles, y_train, shapelet_size, scoring_function):
        """
        Extracts a (greedy) optimal shapelet.
        """
        # Iterate through all samples in profiles
        for sample_idx in range(profiles.shape[0]):
            # The minimum distances of the given samples candidates to all other samples - shape: n_samples, n_candidates
            sample = profiles[sample_idx]
            # Iterate through all candidate distances of the given sample
            for candidate_idx in range(sample.shape[0]):
                # The minimum distances of a candidate to all other samples
                candidate = sample[candidate_idx,:]
                # Add features if other shapelets have been extracted
                features = self.get_features(candidate)
                # Score candidate
                score, margin = scoring_function(features, y_train)
                self.shapelets.append((sample_idx, candidate_idx, score, margin, shapelet_size, candidate))
                
    def get_features(self,candidate):
        """
        If shapelets have been extracted, returns the min distances of all already extracted shapelets + candidate as features
        """
        if len(self.features) == 0:
            return np.array(candidate).reshape((candidate.shape[0],1))
        return np.array([candidate]+self.features).T

    def retrieve_top_shapelet(self, X_train):
        # Sort shapelets according to info gain descending
        self.shapelets.sort(key=lambda x: (x[2],x[3]), reverse=True)

        top_shapelet_found = False
        for sample_idx, candidate_idx, score, margin, shapelet_size, candidate in self.shapelets:
            # If the correct number of shapelets was found, break out of loop
            if top_shapelet_found:
                break
            # Check if sample index of candidate in exclusion zone samples
            if sample_idx not in self.exclusion_zone.keys():
                self.exclusion_zone[sample_idx] = []
            else:
                # Otherwise check if candidate index is in exclusion zone of sample
                if candidate_idx in self.exclusion_zone[sample_idx]:
                    continue
            # Extend exclusion zone
            self.exclusion_zone[sample_idx].extend(list(range(candidate_idx - shapelet_size, candidate_idx+shapelet_size)))
            # Add profile features to features
            self.features.append(candidate)
            # Add shapelet to top shapelets
            self.top_shapelets.append((sample_idx, candidate_idx, score, margin, shapelet_size, X_train[sample_idx,candidate_idx:candidate_idx+shapelet_size]))
            # Signal shapelet found
            top_shapelet_found = True


    def calculate_infogain(self, candidate, y_train, target_class = 1):
        """
        Given a 1-d array, calculates the infogain according the labels y_train.
        """
        # Initialize decision tree classifier
        clf = DecisionTreeClassifier(random_state=0, criterion='entropy', max_depth=1)
        # Fit decision tree
        clf.fit(candidate, y_train)
        self.clf=clf
        # Get entropy before best split
        entropy_before = clf.tree_.impurity[0]
        # Get entropy after best split
        entropy_after = clf.tree_.value.sum(-1)[1]/clf.tree_.value.sum(-1)[0] * clf.tree_.impurity[1] + \
            clf.tree_.value.sum(-1)[2]/clf.tree_.value.sum(-1)[0] * clf.tree_.impurity[2]

        # Calculate margin
        if len(set(y_train)) != 2:
            print(f"There is something wrong with the number of labels! The number o labels is {len(set(y_train))}.")
            raise ValueError
        label_avgs = [candidate[y_train==label].mean() for label in set(y_train)]
        margin = abs(label_avgs[0]-label_avgs[1])
        # Return information gain
        return entropy_before - entropy_after, margin

    @staticmethod
    def standardize_samples_candidates(samples, axis=2):
        """
        Standardized each shapelet candidate (after windowing).
        """
        return (samples-np.expand_dims(samples.mean(axis=axis),axis))/np.expand_dims(samples.std(axis=axis),axis)

    

GSS = GreedyShapeletSearch()
GSS.get_top_k_shapelets(X_train, y_train, scoring_function=fit_svm, n_shapelets=2, shapelet_min_size=30, shapelet_max_size=40)
# ST.main_event_loop(X_train, y_train, n_shapelets=5, shapelet_size=38)
# GSS.main_event_loop(X_train, y_train, scoring_function=GSS.calculate_infogain, shapelet_min_size=30, shapelet_max_size=40)

In [ ]:
# GSS.top_shapelets

In [ ]:
# import time
# result_mapping = {}
# time_mapping = {}
# for i in range(5):
#     start = time.time()
#     GSS = GreedyShapeletSearch()
#     GSS.get_top_k_shapelets(X_train, y_train, scoring_function=fit_svm, n_shapelets=40, shapelet_min_size=30, shapelet_max_size=40)
#     result_mapping[i] = GSS.top_shapelets
#     time_mapping[i] = time.time()-start

In [ ]:
# import json
# with open('time_mapping.json', 'w') as f:
#     json.dump(time_mapping, f)

In [ ]:
# GSS.top_shapelets
# GSS.

# GSS.shapelets
# GSS.top_shapelets = []
# GSS.top_shapelets.append((8,105,0,0,0,39))


# profiles = GSS.get_candidate_mins(X_train, shapelet_size=39)
# candidate = profiles[8,105,:]
# fit_svm(candidate,y_train)


# plt.plot(candidate[y_train==1],color = 'red')
# plt.plot(candidate[y_train==2],color = 'blue')

# print(max(candidate[y_train==2]))
# print(min(candidate[y_train==1]))

In [9]:
# 
def features_transform(X):
    """
    Transformation pipeline for a new dataset X
    """
    features =  []
    for _, _, _, _, shapelet_size, shapelet in GSS.top_shapelets[:3]:
        # Normalize shapelet
        shapelet_norm = GSS.standardize_samples_candidates(shapelet, axis=0)
        # Window the data
        windowed_test = GSS.rolling_window(X, window=shapelet_size)
        # Normalize the windowed data
        windowed_test_norm = GSS.standardize_samples_candidates(windowed_test)
        # Calculate features for shapelet and X
        shapelet_features = ((windowed_test_norm-shapelet_norm)**2).sum(axis=-1).min(axis=-1)
        features.append(shapelet_features)
    return np.array(features).T

# features = features_transform(X_train, X_train)
# print(features.shape)
# print(features)


def fit_classifier(X_train, y_train, X_test, y_test, classifier, scoring_function):
    """
    Tests the performance of a classifier that is first trained  and then tested according to a specified scoring function.
    """
    # Apply the feature pipeline to the training set and testing set to get the min distances of each shapelet
    features_train = features_transform(X_train)
    features_test = features_transform(X_test)
    # Normalizing min distances
    features_train_norm, features_test_norm = feature_normalization(features_train, features_test)
    classifier.fit(features_train_norm, y_train)
    y_pred = classifier.predict(features_test_norm)
    return scoring_function(y_test, y_pred)

def feature_normalization(features_train, features_test):
    features_train_norm = (features_train-features_train.mean(axis=0))/features_train.std(axis=0)
    features_test_norm = (features_test-features_train.mean(axis=0))/features_train.std(axis=0)
    return features_train_norm, features_test_norm


from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

# clf = SVC(kernel='linear')
clf = make_pipeline(StandardScaler(), SVC(kernel='linear',class_weight='balanced'))



fit_classifier(X_train, y_train, X_test, y_test, clf, f1_score)

# def extract_shapelets(X_train, sample_idx, candidate_idx, shapelet_size):
#     """
#     Given a shapelet_size and a given shapelet, computes the profiles 
#     """
#     # Edtract shapelet from data 
#     shapelet = X[sample_idx, candidate_idx:candidate_idx+shapelet_size]
#     shapelets_norm = GSS.standardize_samples_candidates(shapelet.reshape(-1,1),axis=0).T
#     windowed_test = GSS.rolling_window(X, window=shapelet_size)
#     windowed_test_norm = GSS.standardize_samples_candidates(windowed_test)
#     feature = (windowed_test_norm-shapelets_norm).sum(axis=-1).min(axis=-1)
#     return feature


0.9387755102040817

In [5]:



def fit_svm(X,Y, target_class=1):
    """
    Fitting a SVM and returning the f1 score and the calculated margin.
    """
    # Initialize the classifier
    clf = SVC(kernel='linear', class_weight='balanced')
    # Adjust the dimensions of X if necessary
    if len(X.shape) == 1:
       X = X.reshape(-1, 1) 
    # Normalize the input data
    X_norm = (X-X.mean(axis=0))/X.std(axis=0)
    # Fit SVM
    clf.fit(X_norm, Y)
    # Calculate margin
    margin = 1 / np.sqrt(np.sum(clf.coef_ ** 2))
    # Predict
    Y_pred = clf.predict(X_norm)
    # Calculate info gain
    entropy_before = calculate_entropy(Y)
    # Start with a 'pure' entropy 
    entropy_after = 0
    # Iterating through classes
    for label in set(Y):
        # Retrieve the true labels of all instances classified as 'label'
        partial_data = Y[Y_pred == label]
        # Add the weighted entropy to the entropy after
        entropy_after += len(partial_data)/len(Y_pred) * calculate_entropy(partial_data)

    return entropy_before-entropy_after, margin

def calculate_entropy(data):
    """
    Helper function to calculate the entropy of a data set.
    """
    pd_series = pd.Series(data)
    counts = pd_series.value_counts()
    entropy = sps.entropy(counts, base=2)
    return entropy


In [ ]:


print(calculate_entropy(y_train))
print(np.sum(entr(y_train))/np.log(2))
print( -np.sum(y_train*np.log2(y_train)))

In [ ]:
clf = make_pipeline(StandardScaler(), SVC(kernel='linear',class_weight='balanced'))
clf['svc']

In [ ]:
clf = DecisionTreeClassifier(random_state=0, criterion='entropy', max_depth=1)
# Fit decision tree
clf.fit(X_train[:,0].reshape(50,1), y_train)
# Get entropy before best split
entropy_before = clf.tree_.impurity[0]

print(entropy_before)
